In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE64599"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE64599"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE64599.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE64599.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE64599.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The Role of Interleukin-23 in the Early Development of Emphysema in HIV1+ Smokers"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['smoking status: smoker'], 1: ['disease state: HIV+', 'disease state: HIV-'], 2: ['cell type: alveolar macrophage']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyzing dataset GSE64599 for the trait: Chronic_obstructive_pulmonary_disease_(COPD)

# 1. Determine gene expression data availability
# Based on the background information, this appears to be a study about HIV+ smokers and emphysema
# However, there is no explicit indication that it contains gene expression data
# The series is described as a "SuperSeries composed of SubSeries"
# Without clear evidence of gene expression data, we'll set is_gene_available to False
is_gene_available = False

# 2. Variable availability and data type conversion

# 2.1 Data Availability
# From the sample characteristics dictionary:
# Row 0: 'smoking status: smoker' - all are smokers, so this is a constant feature
# Row 1: 'disease state: HIV+', 'disease state: HIV-' - this could be related to our trait (COPD)
# Row 2: 'cell type: alveolar macrophage' - all are the same cell type, so this is a constant feature

# For COPD trait, we might infer it from the HIV status, but this is not a direct indication of COPD
# The background mentions "emphysema" which is related to COPD, but we don't have explicit COPD status
trait_row = None  # No clear indication of COPD status

# Age and gender are not present in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
# Since none of the variables are available, we'll define placeholder conversion functions

def convert_trait(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # No clear mapping for COPD from the available data
    return None

def convert_age(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)  # Convert to continuous numeric value
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    # Binary coding: female=0, male=1
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE64599/GSE64599-GPL570_series_matrix.txt.gz
Gene data shape: (54675, 34)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine gene identifiers to determine type
# The identifiers like '1007_s_at', '1053_at', etc. are Affymetrix probe IDs
# from GPL570 platform (HG-U133_Plus_2), not standard human gene symbols
# They need to be mapped to gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain the probe IDs and gene symbols
# From the preview, we can see:
# - 'ID' contains probe IDs like '1007_s_at' which match the gene expression data index
# - 'Gene Symbol' contains gene symbols like 'DDR1 /// MIR4640'

# 2. Get gene mapping dataframe using the appropriate columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping shape: {gene_mapping.shape}")

# Preview the mapping to confirm it's correct
print("\nGene mapping preview (first 5 rows):")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nConverted gene expression data shape: {gene_data.shape}")

# 4. Preview the gene expression data after mapping
print("\nGene expression data preview (first 5 genes):")
print(preview_df(gene_data, n=5))

# 5. Update the gene availability flag based on the successful gene mapping
is_gene_available = True


Gene mapping shape: (45782, 2)

Gene mapping preview (first 5 rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Converted gene expression data shape: (21278, 34)

Gene expression data preview (first 5 genes):
{'GSM1575085': [92.1398, 79.0887, 65.27392, 5640.59, 95.5121], 'GSM1575086': [87.1394, 29.9709, 146.892, 10535.9921, 66.9938], 'GSM1575087': [136.88, 26.5951, 157.2644, 5956.396000000001, 72.5051], 'GSM1575088': [169.799, 97.2438, 117.82857, 4601.3861, 78.371], 'GSM1575089': [149.572, 10.3971, 98.7029, 10920.6929, 102.916], 'GSM1575090': [137.513, 86.5203, 93.7718, 2494.5275, 68.8274], 'GSM1575091': [110.333, 100.256, 174.49509999999998, 8309.0367, 51.4439], 'GSM1575092': [180.974, 14.1813, 122.40554, 4087.6662, 66.1764], 'GSM1575093': [198.452, 20.6847, 38.26969, 19431.861, 63.7477], 'GSM1575094': [292.859, 98.5018, 186.88909999999998, 1693.569, 179.683], 'GSM1575095': [263.956, 126.756, 29.742820000000002, 2644.6365, 86.2573], 'GSM1575096': [281.398, 33.0788, 125.3596, 14857.9386, 128.955], 'GSM1575097': [110.974, 114.035, 119.18323000000001, 7373.889, 126.573], 'GSM1575098': [184.895, 2

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Save normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (is_trait_available = False),
# we can't create clinical features or perform linking with trait data

# Update gene availability flag based on successful gene mapping
is_gene_available = True
is_trait_available = False  # As determined in Step 2

# Create a note about the dataset
note = "Dataset contains gene expression data from alveolar macrophages of HIV+ and HIV- smokers. No COPD trait information is available."

# Create a minimal dataframe with proper structure for the validation function
dummy_df = pd.DataFrame(index=normalized_gene_data.index[:5], columns=normalized_gene_data.columns[:5])

# Conduct final quality validation
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=False,  # Set to False when no trait data
    df=dummy_df,      # Provide a non-empty DataFrame with structure
    note=note
)

print(f"Dataset usability status: {is_usable}")

Normalized gene data shape: (19845, 34)


Normalized gene data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE64599.csv
Dataset usability status: False
